In [2]:
import numpy as np
import tensorflow as tf

In [4]:
def block(inputs, filters):
    conv1 = tf.layers.conv2d(inputs,
                            filters,
                            4,
                            padding="same",
                            activation="relu",
                            name="conv{}_1".format(filters))
    conv2 = tf.layers.conv2d(conv1,
                            filters,
                            4,
                            padding="same",
                            activation="relu",
                            name="conv{}_2".format(filters))
    conv3 = tf.layers.conv2d(conv2,
                            filters,
                            4,
                            padding="same",
                            activation="relu",
                            name="conv{}_3".format(filters))
    pool = tf.layers.max_pooling2d(conv3,
                            pool_size=2,
                            strides=2,
                            name="pool{}".format(filters))
    return pool

In [8]:
def architecture(inputs, filter_size, output_size):
    
    with tf.variable_scope('CovNet'):
        for filter_size in enumerate(filter_size):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                      inputs = block(inputs, filter_size)
        
        with tf.name_scope("conv-dense"):
            dense1 = tf.layers.dense(
                inputs,
                4096,
                activation='relu',
                name='dense1',
            )
            dense2 = tf.layers.dense(
                dense1,
                4096,
                activation='relu',
                name='dense2',
            )
            net = tf.layers.dense(
                dense2,
                output_size,
                activation='softmax',
                name='softmax',
            )

    return net

In [9]:
def get_train_op_fn(loss, params):
    return tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        optimizer=tf.train.AdamOptimizer,
        learning_rate=params.learning_rate
)

In [10]:
def get_eval_metric_ops(labels, predictions):
    return {
        'Accuracy': tf.metrics.accuracy(
            labels=labels,
            predictions=predictions,
            name='accuracy')
    }

In [12]:
def model_fn(features, labels, mode, params):
    filter_size = [64,128,256,512,512]    
    output_size = labels.shape
    is_trainig = mode == ModeKeys.TRAIN
    logits = architecture(features, filter_size, output_size)
    predictions = tf.argmax(logits, axis = -1)
    
    loss = None
    train_op = None
    eval_metric_ops = {}
    if mode != ModeKeys.INFER:
        loss = tf.losses.log_loss(
            labels=tf.cast(labels, tf.int32),
            logits=logits)
        train_op = get_train_op_fn(loss, params)
        eval_metric_ops = get_eval_metric_ops(labels, predictions)
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops
    )

In [13]:
def get_estimator(run_config, params):
    return tf.estimator.Estimator(
        model_fn=model_fn,  # First-class function
        params=params,  # HParams
        config=run_config  # RunConfig
    )

In [14]:
class IteratorInitializerHook(tf.train.SessionRunHook):

    def __init__(self):
        super(IteratorInitializerHook, self).__init__()
        self.iterator_initializer_func = None

    def after_create_session(self, session, coord):
        self.iterator_initializer_func(session)

In [15]:
def get_train_inputs(batch_size, images, labels):
    
    iterator_initializer_hook = IteratorInitializerHook()

    def train_inputs():
        with tf.name_scope('Training_data'):
            # Get Mnist data
            images = images.reshape([-1, 28, 28, 1])
            # Define placeholders
            images_placeholder = tf.placeholder(
                images.dtype, images.shape)
            labels_placeholder = tf.placeholder(
                labels.dtype, labels.shape)
            # Build dataset iterator
            dataset = tf.contrib.data.Dataset.from_tensor_slices(
                (images_placeholder, labels_placeholder))
            dataset = dataset.repeat(None)  # Infinite iterations
            dataset = dataset.shuffle(buffer_size=10000)
            dataset = dataset.batch(batch_size)
            iterator = dataset.make_initializable_iterator()
            next_example, next_label = iterator.get_next()
            # Set runhook to initialize iterator
            iterator_initializer_hook.iterator_initializer_func = \
                lambda sess: sess.run(
                    iterator.initializer,
                    feed_dict={images_placeholder: images,
                               labels_placeholder: labels})
            # Return batched (features, labels)
            return next_example, next_label

    # Return function and hook
    return train_inputs, iterator_initializer_hook


In [ ]:
def experiment_fn(run_config, params):
    
    run_config = run_config.replace(save_checkpoints_steps=params.min_eval_frequency)
    estimator = get_estimator(run_config, params)
    
    filename_Que = tf.train.string_input_producer(tf.train.match_filenames_once("./images/*.jpg"))
    
    